In [31]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.integrate import simps

# Lendo arquivo das waveforms

#### O arquivo original já foi parcialmente modificado em relação a linhas e colunas

In [2]:
%%time
'''Lendo arquivo das waveforms'''

waveform = pd.read_csv(
    '5555_eventos-edit.csv', 
    index_col = 0
                      ) # importa como waveform vs sample
waveform

Wall time: 2.76 s


,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,...,event_5546,event_5547,event_5548,event_5549,event_5550,event_5551,event_5552,event_5553,event_5554,time
0,56.0,56.0,56.0,53.0,55.0,55.0,54.0,54.0,54.0,54.0,...,54.0,53.0,53.0,51.0,51.0,51.0,51.0,51.0,51.0,0
1,55.0,55.0,55.0,53.0,55.0,55.0,54.0,54.0,54.0,54.0,...,52.0,52.0,52.0,52.0,51.0,51.0,51.0,51.0,51.0,1
2,53.0,53.0,53.0,51.0,53.0,53.0,55.0,55.0,55.0,55.0,...,53.0,53.0,53.0,50.0,55.0,55.0,55.0,55.0,55.0,2
3,53.0,53.0,53.0,52.0,53.0,53.0,54.0,54.0,54.0,54.0,...,53.0,53.0,53.0,50.0,55.0,55.0,55.0,55.0,55.0,3
4,54.0,54.0,54.0,53.0,55.0,55.0,55.0,55.0,55.0,55.0,...,51.0,53.0,53.0,52.0,54.0,54.0,54.0,54.0,54.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,53.0,53.0,53.0,54.0,55.0,55.0,22.0,22.0,22.0,22.0,...,54.0,50.0,50.0,50.0,52.0,52.0,52.0,52.0,52.0,2495
2496,55.0,55.0,55.0,54.0,56.0,56.0,14.0,14.0,14.0,14.0,...,50.0,51.0,51.0,53.0,51.0,51.0,51.0,51.0,51.0,2496
2497,52.0,52.0,52.0,53.0,53.0,53.0,37.0,37.0,37.0,37.0,...,53.0,51.0,51.0,53.0,52.0,52.0,52.0,52.0,52.0,2497
2498,52.0,52.0,52.0,53.0,54.0,54.0,51.0,51.0,51.0,51.0,...,52.0,53.0,53.0,52.0,52.0,52.0,52.0,52.0,52.0,2498


# Construindo os métodos para a análise de dados

#### 1. Peak_finder 

Auxilia a encontrar todos os picos de uma waveform. No caso, encontra dois picos.

In [3]:
def peak_finder(_s): # peak_finder se aplica em data frames com valores do eixo y    
    '''
    Essa função retorna os valores no eixo x
    '''

    _s *= -1 # CUIDADO: aparentemente, isso muda o df. Tenha certeza que irá desfazer isso depois
    
    # get the actual peaks
    peaks, _ = find_peaks(_s, height = 50) # height é um parâmetro decidido
    
    # print(peaks)
    
    # multiply back for plotting purposes
    _s *= -1
    
    
    return(peaks)

#### 2. delta_signal

Encontra as diferenças entre elementos lado-a-lado. Serve para avaliar quando estamos começando ou terminando um pulso.

In [4]:
def delta_x(s): # insira uma series na entrada; o valor retornado está em coordenadas do eixo x
    VA_derivada_baseLine = 5  #   flutuação/amplitude máxima da base-line;
                              #   valor arbirtrário para saber se o número em questão está fora da 
                              # base-line; não é exatamente a derivada
    lst = []
    for i in range( len(s) - 1 ): # i = index da series
        if abs(s[i] - s[i+1]) > VA_derivada_baseLine:
            lst.append(i) 
    #print(lst)
        
    return (lst) # o valor retornado está em coordenadas do eixo x

#### 3. contorno_pulso

Encontra os valores dos dois pulsos em cada uma das waveforms. Utiliza as duas funções anteriores. 
Funciona da seguinte maneira: encontra os picos e delimita os entornos. Daí, guarda os pontos interiores do pulso e os devolve.

In [5]:
def contorno_pulso(_s, VA_1, VA_2): # recebe uma Series do pandas para começar
    
    
    '''
    Define um recorte de onde se deve buscar um pulso e os seus delimitadores
    VA_1 e VA_2 são variáveis arbitrárias para definir a largura do pulso 
    '''
    

    peak = peak_finder(_s) # encontrar os picos da waveform
    
    s1 = _s[ (peak[0] - VA_1):(peak[0] + VA_2) ] # recortar envolta dos picos
    s2 = _s[ (peak[1] - VA_1):(peak[1] + VA_2) ]
    
    df1 = pd.DataFrame( dict(s1 = s1) ).reset_index() # cria um Data Frame com os valores do recorte
    df1.columns = ['time', 's1']                      # renomeia a coluna do data frame
    df2 = pd.DataFrame( dict(s1 = s2) ).reset_index()
    df2.columns = ['time', 's2']
    
    
    

    '''
    Calcular a "derivada" em cada ponto no entorno, para saber os limitantes do pulso
    Ao terminar, retornar o data frame que contem os valores limitantes do contorno do pulso
    '''  

    indexLim_1 = delta_x( df1['s1'] ) # índices limitantes
    indexLim_2 = delta_x( df2['s2'] )
    
    # redefine os valores para apenas os limitantes do data frame
    df1 = df1.iloc[    [  indexLim_1[0], indexLim_1[-1]  ]    ] 
    df2 = df2.iloc[    [  indexLim_2[0], indexLim_2[-1]  ]    ] 
    
    # print(df2) # series marcada pelas colunas 
    
    
    # da Series original, temos agora o contorno do pulso
    s1 = _s[ df1['time'].iloc[0] : df1['time'].iloc[1]+1 ] # soma 1 para incluir o último termo
    s2 = _s[ df2['time'].iloc[0] : df2['time'].iloc[1]+1 ] 
    
    # print(s2)
    
    pulsos = s1, s2
    
    
    
    
    
    
    return(pulsos) # retorna os dois contornos, um de cada pulso

#### integral_pulso

Calcula a integral do contorno do pulso da waveform utilizando a integração numérica de Simpson.

In [6]:
def integral_pulso(tupla_de_Series): # df = ['time', 'pulso_1', 'pulso_2']
    
    s1, s2  =   tupla_de_Series[0], tupla_de_Series[1]
    
    integral_pulso = [ 
        simps(s1, dx = 1),
        simps(s2, dx = 1)
                ]
    
    return(integral_pulso)

In [32]:
'''
Definimos essa função que retorna uma sample da base line da waveform
'''

def baseLine_sample(_s): # recebe uma series do pandas
    
    _ = peak_finder(_s)
    
    '''Pega todos os elementos até o início do primeiro pulso; intervalo exclusivo à direita'''
    
    if len(_) != 0: # pode acontecer de que o peak_finder não 
        
        x_inicio_pulso_0 = _[0] - VA_1 # definido arbitrariamente
        # print(x_inicio_pulso_0)
    
        sample  = evento.iloc[:x_inicio_pulso_0]
        
    else:
        #print(f'problema em {_s.name}')
        sample = np.nan

    return( sample ) # este elemento é uma Series

# Análise de dados

### Cópia a janela de busca

Cria uma cópia das waveforms para poder transladar o base line sem modificar os dados originais;
Define a janela onde procuraremos pelos valores do contorno do pulso

In [19]:
df = waveform.copy(deep = True)

VA_1 = 20 # VA_1 e VA_2 são variáveis arbitrárias para definir o majorante da largura dos pulsos
VA_2 = 70

df.head()

,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,...,event_5546,event_5547,event_5548,event_5549,event_5550,event_5551,event_5552,event_5553,event_5554,time
0,56.0,56.0,56.0,53.0,55.0,55.0,54.0,54.0,54.0,54.0,...,54.0,53.0,53.0,51.0,51.0,51.0,51.0,51.0,51.0,0
1,55.0,55.0,55.0,53.0,55.0,55.0,54.0,54.0,54.0,54.0,...,52.0,52.0,52.0,52.0,51.0,51.0,51.0,51.0,51.0,1
2,53.0,53.0,53.0,51.0,53.0,53.0,55.0,55.0,55.0,55.0,...,53.0,53.0,53.0,50.0,55.0,55.0,55.0,55.0,55.0,2
3,53.0,53.0,53.0,52.0,53.0,53.0,54.0,54.0,54.0,54.0,...,53.0,53.0,53.0,50.0,55.0,55.0,55.0,55.0,55.0,3
4,54.0,54.0,54.0,53.0,55.0,55.0,55.0,55.0,55.0,55.0,...,51.0,53.0,53.0,52.0,54.0,54.0,54.0,54.0,54.0,4


### Translação da base-line

É interessante transladar os pontos e fixar a base line no 0. Podemos fazer isso somando -50 em cada um dos valores de toda a coleção de waveforms. Isso servirá para evitar problemas de 'height' no peak_finder()

Análise estatística do base-line:
No caso, tomaremos todos os dados antes do primeiro pulso de cada waveform e consideraremos o base-line como o mesmo para todos os waveforms

#### Método:

Para encontrar a base line, selecionaremos a parte mais à esquerda, antes do pulso. 
Esse pedaço é definido como fora de um valor arbitrário em torno do primeiro pico da waveform, que acaba não sofrendo com a questão da base line. Definimos desse jeito porque a função para calcular o contorno do pulso, além de demorar para ser computada, sofre com as questões do peak_finder acabar encontrando apenas um elemento


In [36]:
%%time
baseLines = [] # é uma lista de series
for i in range( df.shape[1] - 1 ):
    evento = df[ df.columns[i] ]
    baseLines.append(baseLine_sample(evento))
#baseLines.dropna()

Wall time: 5.74 s


nan

In [37]:
# baseLines = pd.concat(baseLines)
baseLines[30]


nan

In [39]:
pd.concat(baseLines, drop)

TypeError: cannot concatenate object of type '<class 'float'>'; only Series and DataFrame objs are valid

In [ ]:
plt.plot( df.index , df.event_30 )

In [ ]:
'''
Alterando os valores e recolocando na base line = 0
'''

df.iloc[ :  , :waveform.shape[1]-1 ] -= baseLines.mean() # soma -50 em todo o df, exceto pela última coluna 'time'
df

In [ ]:
contorno_pulso(waveform.event_75)

In [ ]:
%%time
a = df[df.columns[190]]
integral_pulso(contorno_pulso(a))

In [ ]:
"""
%%time

'''
Loop para integrar os pulsos em todo o Data Frame
'''

integrais = []
for i in range( waveform.shape[1] - 1 ): # loop para integrar os pulsos em todo o Data Frame
    evento = df[df.columns[i]]
    integrais.append( integral_pulso(contorno_pulso(evento)) )
    
"""

In [ ]:
contorno_pulso(waveform.event_75)[1]

In [ ]:
plt.plot( contorno_pulso(waveform.event_75)[1].index , contorno_pulso(waveform.event_75)[1] )

In [ ]:
plt.plot(waveform.time.iloc[50:150], waveform.event_1254.iloc[50:150])

In [ ]:
plt.plot( contorno_pulso(waveform.event_3)[1].index , contorno_pulso(waveform.event_3)[1] )

In [ ]:
aux = []
for i in range(100): #waveform.shape[1] - 1
    evento = df[df.columns[i]]
    aux.append( [ len(contorno_pulso(evento)[0]), len(contorno_pulso(evento)[1]) ] )
#aux

In [ ]:
plt.plot( waveform.time.iloc[300:400], waveform.event_3.iloc[300:400] )

In [ ]:
picos = []
for i in range(100): #waveform.shape[1] - 1
    evento = df[df.columns[i]]
    picos.append( peak_finder(evento) )

picos_em_x = pd.DataFrame(picos, columns = ['pico_x_0', 'pico_x_1'])
picos_em_x['pico_x_0']

In [ ]:
waveform[waveform.columns[0]].iloc[94]

## Fazer análise estatística da base line
## Calcular a integral da janela de -20/+80
## Plotar espectro de pulso e espectro de carga